1. word2idx,idx2word로 vocab만둘기  
2. randomly embedding  
3. LSTM(셀? nn?) 층 만들기  

In [1]:
import os
import re
import numpy as np
from collections import OrderedDict

In [2]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
path ='./data'
BATCH, D_in, H, D_out = 2,10,2,2

## 데이터 로드

In [4]:
import pickle
with open(os.path.join(path,'tagged_train.dat'), 'rb') as f:
    tagged_list = pickle.load(f)

In [5]:
tagged_list[:10]

[[['<SOS>',
   'fast/RB',
   'methods/NNS',
   'for/IN',
   'recovering/VBG',
   'sparse/JJ',
   'parameters/NNS',
   'in/IN',
   'linear/JJ',
   'low/JJ',
   'rank/NN',
   'models/NNS'],
  ['in/IN',
   'this/DT',
   'paper/NN',
   ',/,',
   'we/PRP',
   'investigate/VBP',
   'the/DT',
   'recovery/NN',
   'of/IN',
   'a/DT',
   'sparse/JJ',
   'weight/NN',
   'vector/NN',
   'parameters/NNS',
   'vector/VBP',
   'from/IN',
   'a/DT',
   'set/NN',
   'of/IN',
   'noisy/JJ',
   'linear/JJ',
   'combinations/NNS',
   './.',
   'however/RB',
   ',/,',
   'only/RB',
   'partial/JJ',
   'information/NN',
   'about/IN',
   'the/DT',
   'matrix/NN',
   'representing/VBG',
   'the/DT',
   'linear/JJ',
   'combinations/NNS',
   'is/VBZ',
   'available/JJ',
   './.',
   'assuming/VBG',
   'a/DT',
   'low-rank/JJ',
   'structure/NN',
   'for/IN',
   'the/DT',
   'matrix/NN',
   ',/,',
   'one/CD',
   'natural/JJ',
   'solution/NN',
   'would/MD',
   'be/VB',
   'to/TO',
   'first/VB',
   'apply/V

In [6]:
title = list(map(lambda x: tagged_list[x][0], range(len(tagged_list))))
abstract = list(map(lambda x: tagged_list[x][1], range(len(tagged_list))))

## w2i, i2w 생성

In [7]:
bow = list(set(re.sub("',|'|\[|\]|", "", str(tagged_list)).split(' ')))

In [8]:
w2i = dict(zip(bow, range(6,len(bow)+6)))
w2i['<PAD>'] = 0
w2i['<UNK>'] = 1
w2i['[SOS]'] = 2
w2i['<EOS>'] = 3
w2i['<SEP>'] = 4
w2i[',/,'] = 5
# w2i = {val:idx for idx, val in enumerate(tagged_list)}
i2w = OrderedDict(sorted(zip(w2i.values(), w2i.keys())))

## randomly embedding

In [9]:
torch.random.manual_seed(0)
embedict = torch.cat((torch.zeros((1,D_in)), torch.randn(len(w2i)-1,D_in)),0)

In [10]:
embedict[:10]

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-1.1258e+00, -1.1524e+00, -2.5058e-01, -4.3388e-01,  8.4871e-01,
          6.9201e-01, -3.1601e-01, -2.1152e+00,  3.2227e-01, -1.2633e+00],
        [ 3.4998e-01,  3.0813e-01,  1.1984e-01,  1.2377e+00,  1.1168e+00,
         -2.4728e-01, -1.3527e+00, -1.6959e+00,  5.6665e-01,  7.9351e-01],
        [ 5.9884e-01, -1.5551e+00, -3.4136e-01,  1.8530e+00,  7.5019e-01,
         -5.8550e-01, -1.7340e-01,  1.8348e-01,  1.3894e+00,  1.5863e+00],
        [ 9.4630e-01, -8.4368e-01, -6.1358e-01,  3.1593e-02, -4.9268e-01,
          2.4841e-01,  4.3970e-01,  1.1241e-01,  6.4079e-01,  4.4116e-01],
        [-1.0231e-01,  7.9244e-01, -2.8967e-01,  5.2507e-02,  5.2286e-01,
          2.3022e+00, -1.4689e+00, -1.5867e+00, -6.7309e-01,  8.7283e-01],
        [ 1.0554e+00,  1.7784e-01, -2.3034e-01, -3.9175e-01,  5.4329e-01,
         -3.9516e-01, -4.4622e-0

In [11]:
embedict[0].view(1,D_in)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [12]:
MAX_LEN = 10

def embedding(text): # vec = abstract or title 한개씩
    needpad = MAX_LEN-len(text)
    if needpad > 0:
        text.extend(['<PAD>']*needpad)
    embedded = []
    for token in text[:MAX_LEN]:
        embedded.append(embedict[w2i[token]].view(1,D_in))
    return torch.cat(embedded,0)


In [54]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1, num_layers=2):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        # Define the LSTM cell
        self.lstm = nn.LSTMCell(self.input_dim, self.hidden_dim)
        self.hx = torch.randn(MAX_LEN, H)
        self.cx = torch.randn(MAX_LEN, H)
        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)
    
    def init_hidden(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layars, self.batch_size, self.hidden_dim))
    def forward(self, x):
        output = []
        for i in range(self.num_layers):
            self.hx, self.cx = self.lstm(x[i], (self.hx, self.cx))
            output.append(hs)
        y_pred = self.linear(output[-1].view(self.batch_size, -1))
        return y_pred.view(-1)

In [55]:
model = LSTM(10, 2, 5, 1, 2)

In [56]:
loss_fn = torch.nn.MSELoss(size_average = False)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [57]:
X_train = y_train = []
for i in abstract[:10]:
    X_train.append(embedding(i))
XX = torch.cat(X_train, 0)
for i in title[:10]:
    y_train.append(embedding(i))
yy = torch.cat(y_train,0)

In [58]:
XX.shape

torch.Size([100, 10])

In [59]:
num_epochs = 10
hist = np.zeros(num_epochs)
for t in range(num_epochs):
    model.zero_grad()
    y_pred = model(XX)
    loss = loss_fn(y_pred, yy)
    print("Epoch",t,"MSE:",loss.item())
    hist[t] = loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)